In [1]:
import pandas as pd
import os
import io
import datetime

In [2]:
start_time = '2014-04-01'
# 1. Convert the_day string to a datetime object
date_obj = datetime.datetime.strptime(start_time, '%Y-%m-%d')
end_time = date_obj + datetime.timedelta(days=6)
print(end_time)
#end_time = '2014-04-07'
one_week_data = pd.read_csv('one_week_data.csv')
one_week_data['Timestamp'] = pd.to_datetime(one_week_data['Timestamp'])
# one_week_data.head(5)

2014-04-07 00:00:00


In [3]:
the_day = '2014-04-05'

# 1. Convert the_day string to a datetime object
date_obj = datetime.datetime.strptime(the_day, '%Y-%m-%d')

# 2. Calculate 9 PM of the given date
nine_pm_current_day = date_obj.replace(hour=21, minute=0, second=0, microsecond=0)

# 3. Calculate 9 PM of one day earlier
one_day_earlier = date_obj - datetime.timedelta(days=1)
nine_pm_one_day_earlier = one_day_earlier.replace(hour=21, minute=0, second=0, microsecond=0)

# 4. Get the timestamps (Unix timestamps)
timestamp_nine_pm_one_day_earlier = nine_pm_one_day_earlier.timestamp()
timestamp_nine_pm_current_day = nine_pm_current_day.timestamp()

print(f"Date used: {the_day}")
print(f"9 PM one day earlier ({nine_pm_one_day_earlier}): {timestamp_nine_pm_one_day_earlier}")
print(f"9 PM of the date ({nine_pm_current_day}): {timestamp_nine_pm_current_day}")

Date used: 2014-04-05
9 PM one day earlier (2014-04-04 21:00:00): 1396616400.0
9 PM of the date (2014-04-05 21:00:00): 1396702800.0


In [4]:
one_day_data = one_week_data[(one_week_data['Timestamp'] >= nine_pm_one_day_earlier) 
    & (one_week_data['Timestamp'] <= nine_pm_current_day)]

In [5]:
# Filter for only motion sensors
df_motion = one_day_data[(one_day_data['SensorType'] == 'Control4-Motion') & (one_day_data['Value'] == 'ON')].copy()
# Sort by timestamp to ensure correct order
# df_motion = df_motion.sort_values('Timestamp').set_index('Timestamp')
df_motion = df_motion.sort_values('Timestamp')

In [6]:
inactivity_threshold_minutes = 45
night_start_hour= 23
night_end_hour = 7
night = '2014-04-04'   # just one night

night_window_start = pd.to_datetime(f"{night} {night_start_hour}:00:00")
# night_window_start = pd.to_datetime(f"2014-04-05 1:33:00")
night_window_end = night_window_start + pd.Timedelta(hours=(24 - night_start_hour + night_end_hour))

# If night_events is a slice of another DataFrame, make a copy first
night_events = df_motion[(df_motion['Timestamp'] >= night_window_start) & (df_motion['Timestamp'] <= night_window_end)].copy()

# night_events = night_events.copy()  # fix the

if len(night_events) < 2:
    print(f"night events:{len(night_events)}")

In [7]:
night_events['end_time'] = night_events['Timestamp'].shift(-1)
night_events['gap_to_next_event_mins'] = (night_events['end_time'] - night_events['Timestamp']).dt.total_seconds() / 60
# # Find the first event that is followed by a long period of inactivity
sleep_onset_events = night_events[night_events['gap_to_next_event_mins'] >= inactivity_threshold_minutes]

if sleep_onset_events.empty:
    print("No sleep period detected for this night.\n")
else:
    print(sleep_onset_events)

      Unnamed: 0 SensorID  Attribute1 Attribute2 Value       SensorType  \
3231     6435040     M004  LivingRoom      Chair    ON  Control4-Motion   
3245     6435054     M004  LivingRoom      Chair    ON  Control4-Motion   

                      Timestamp                   end_time  \
3231 2014-04-05 00:08:27.058745 2014-04-05 01:29:48.294037   
3245 2014-04-05 01:33:11.467297 2014-04-05 06:37:46.316029   

      gap_to_next_event_mins  
3231               81.353922  
3245              304.580812  


In [17]:
# first_sleep_onset_event = sleep_onset_events.iloc[0]
sleep_start_time = sleep_onset_events.iloc[0]['Timestamp']
# first_sleep_end_time = sleep_onset_events.iloc[0]['end_time']

# The main sleep session starts after this event
print(f"Initial sleep detected after event at {first_sleep_end_time.time()}.")

# The final wake-up is the last event in the night sequence
# final_wake_up_time = sleep_onset_events.iloc[-1]['end_time']
final_sleep_start_time = sleep_onset_events.iloc[-1]['Timestamp']
print(final_sleep_start_time)
# Now, find any interruptions *between* the start of sleep and the final wake-up
interruptions = night_events[(night_events['Timestamp']> sleep_start_time) & (night_events['Timestamp'] < final_sleep_start_time)]


Initial sleep detected after event at 01:29:48.294037.
2014-04-05 01:33:11.467297


In [23]:
sleep_onset_events.head(10)

,Unnamed: 0,SensorID,Attribute1,Attribute2,Value,SensorType,Timestamp,end_time,gap_to_next_event_mins
3231,6435040,M004,LivingRoom,Chair,ON,Control4-Motion,2014-04-05 00:08:27.058745,2014-04-05 01:29:48.294037,81.353922
3245,6435054,M004,LivingRoom,Chair,ON,Control4-Motion,2014-04-05 01:33:11.467297,2014-04-05 06:37:46.316029,304.580812


In [18]:
interruptions.head(10)

,Unnamed: 0,SensorID,Attribute1,Attribute2,Value,SensorType,Timestamp,end_time,gap_to_next_event_mins
3239,6435048,M004,LivingRoom,Chair,ON,Control4-Motion,2014-04-05 01:29:48.294037,2014-04-05 01:32:02.787402,2.241556
3241,6435050,M005,DiningRoom,DiningRoom,ON,Control4-Motion,2014-04-05 01:32:02.787402,2014-04-05 01:32:10.085275,0.121631
3243,6435052,M005,DiningRoom,DiningRoom,ON,Control4-Motion,2014-04-05 01:32:10.085275,2014-04-05 01:33:11.467297,1.023034


In [27]:
if not interruptions.empty:
    print(f"Detected {len(interruptions)} motion event(s) during sleep.")
    # print(interruptions)

# Calculate the sum of the 'gap_to_next_event_mins' column
net_sleep_duration = sleep_onset_events['gap_to_next_event_mins'].sum()
final_sleep_end_time = sleep_onset_events.iloc[-1]['Timestamp']
awake_duration_total = interruptions['gap_to_next_event_mins'].sum()
gross_sleep_duration = net_sleep_duration + awake_duration_total

print(sleep_start_time, final_sleep_end_time, gross_sleep_duration, awake_duration_total)



Detected 3 motion event(s) during sleep.
2014-04-05 00:08:27.058745 2014-04-05 01:33:11.467297 389.3209547333333 3.3862210000000004
